## Goal

Build a simple chain within our graph that uses 4 concepts/tools
- Use chat messages in the graph
- Invoking chat models within graph
- Binding tools to out LLM
- Executing tool calls in our graph

In [37]:
# general imports
import sys
import os, getpass
from openai import OpenAI
from utils import *
from IPython.display import Image
from pprint import pprint

# Import langgraph modules to create and visualize the graph
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage 
from typing_extensions import TypedDict
from typing import Literal
from pydantic import BaseModel, Field

# Install the missing package
# %pip install langchain_anthropic
# %pip install langchain_groq
from langchain_groq import ChatGroq

In [3]:
# Let's make sure we have the API key set
GROQ_API_KEY = set_api_key('GROQ_API_KEY')

API key found in .env file for GROQ_API_KEY


#### Messages

Chat models can use `messages`, which captures different roles within a conversation (e.g., "system", "human", "assistant").
Messages also feature one or more content blocks that contain text or potentially multimodal data (e.g., images, audio, video).

<h5>Let's create a list of messages. Each message can be supplied with a few things:</h5>

- `content` - content of the message
- `name` - optionally, who is creating the message
-  `response_metadata` - optionally, a dict of metadata that is oftern specific to each model provider

In [4]:

messages = [AIMessage(content=f'So you wanted to know more starwars conception idea?', name = 'AI')]
messages.extend([HumanMessage(content=f'Yes, I am interested in starwars', name = 'Human')])
messages.extend([AIMessage(content=f'What do you want to know about starwars?', name = 'AI')])
messages.extend([HumanMessage(content=f'What is the starwars universe?', name = 'Human')])


In [5]:
for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: AI

So you wanted to know more starwars conception idea?
================================ Human Message =================================
Name: Human

Yes, I am interested in starwars
================================== Ai Message ==================================
Name: AI

What do you want to know about starwars?
================================ Human Message =================================
Name: Human

What is the starwars universe?


### Chat Models

Chat models use a sequence of message as input and support message roles.

There are many chat models available out there to choose from, in our case, we will use Groq.

We can load the AI model and invoke it with our list of messages.


In [88]:
ai = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY, temperature=0.8, max_retries=2)

In [89]:
result = ai.invoke(messages)
result.pretty_print()

================================== Ai Message ==================================

The Star Wars universe is a vast, fictional galaxy created by George Lucas. It's a sprawling, epic space fantasy that spans thousands of years, numerous planets, and a diverse array of alien species, technologies, and cultures.

The Star Wars universe is set in a galaxy far, far away, where various factions, empires, and alliances have risen and fallen over time. The galaxy is divided into different regions, including:

1. **The Core Worlds**: The central, densely populated region of the galaxy, where the Galactic Republic and later the Galactic Empire were formed.
2. **The Outer Rim**: A peripheral region of the galaxy, home to many independent systems and a hotbed of smuggling and piracy.
3. **The Unknown Regions**: A vast, uncharted area of the galaxy, home to many mysterious and unexplored worlds.

The Star Wars universe is filled with a wide range of characters, including:

* **Jedi and Sith**: Power

In [90]:
result.response_metadata

{'token_usage': {'completion_tokens': 531,
  'prompt_tokens': 90,
  'total_tokens': 621,
  'completion_time': 1.930909091,
  'prompt_time': 0.00608742,
  'queue_time': 0.232749072,
  'total_time': 1.936996511},
 'model_name': 'llama-3.3-70b-versatile',
 'system_fingerprint': 'fp_7b42aeb9fa',
 'finish_reason': 'stop',
 'logprobs': None}

In [91]:
stream = ai.stream(messages)
full = next(stream)
for chunk in stream:
    full += chunk
full.pretty_print()

============================ Aimessagechunk Message ============================

The Star Wars universe, also known as the Star Wars galaxy, is a vast, fictional universe created by George Lucas. It's a sprawling, epic space fantasy that spans thousands of years, numerous planets, and a diverse array of alien species.

The Star Wars universe is set in a distant galaxy, where a variety of planets, star systems, and regions are connected by a network of hyperspace lanes. The universe is home to a multitude of civilizations, including humans, aliens, robots, and other sentient beings.

The Star Wars universe is divided into several key components:

1. **Galactic Government**: The galaxy is governed by various factions, including the Galactic Republic, the Galactic Empire, and the New Republic. Each of these governments has its own system of governance, military, and laws.
2. **Planets and Star Systems**: The Star Wars universe features a vast array of planets, moons, and star systems, ea

<h4> Tools </h4>

Tools are an additional capability provided to LLMs that can be used to interact with outside world.

It could be leveraged to control different parts of the code, call an external API, interact with database, trigger a lambda function, etc.

Tools can be as simple as Python functions or as complex as integrating LLM with RAG for real time analysis.

In [92]:
class GetExponent(BaseModel):
    """
    Raise x to the power of y.
    
    Args:
    x (int): The base number.
    y (int): The exponent number.
    result (int): raise x to the power of y.
    """
    x: int = Field(description='The base number', example=2)
    y: int = Field(description='The exponent number', example=3)
    result: int = Field(description='perform x ** y ', example=8)
    

In [95]:
ai_with_tool = ai.bind_tools([GetExponent])

In [ ]:
tool_call = ai_with_tool.invoke([HumanMessage(content='You are a calculator assistant with all the tools required to perfom mathematical operations. \
                                              What is 2 raised to the power of 3')])

In [100]:
tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ap3m', 'function': {'arguments': '{"x": 2, "y": 3, "result": 8}', 'name': 'GetExponent'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 705, 'total_tokens': 732, 'completion_time': 0.098181818, 'prompt_time': 0.027536548, 'queue_time': -0.126286389, 'total_time': 0.125718366}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_7b42aeb9fa', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-556da27d-d13e-413c-a691-d54846041cf3-0', tool_calls=[{'name': 'GetExponent', 'args': {'x': 2, 'y': 3, 'result': 8}, 'id': 'call_ap3m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 705, 'output_tokens': 27, 'total_tokens': 732})